In [1]:
import numpy as np
import symmray as sr
import quimb.tensor as qtn

In [2]:
Lx = 6
Ly = 6
D = 6
seed = 42

In [3]:
edges = qtn.edges_2d_square(Lx, Ly)
site_info = sr.utils.parse_edges_to_site_info(
    edges,
    D,
    phys_dim=2,
    site_ind_id="k{},{}",
    site_tag_id="I{},{}",
)

In [4]:
peps = qtn.TensorNetwork()
rng = np.random.default_rng(seed)

for site, info in sorted(site_info.items()):

    # bond index charge distribution
    block_indices = [
        sr.BlockIndex({0: d // 2, 1: d // 2}, dual=dual)
        for d, dual in zip(info["shape"][:-1], info["duals"][:-1])
    ]
    # physical index
    p = info['shape'][-1]
    block_indices.append(
        sr.BlockIndex({0: p // 2, 1: p // 2}, dual=info["duals"][-1])
    )

    data = sr.Z2FermionicArray.random(
        block_indices,
        charge=0,
        seed=rng,
    )

    peps |= qtn.Tensor(
        data=data,
        inds=info["inds"],
        tags=info["tags"],
    )

# required to view general TN as an actual PEPS
for i, j in site_info:
    peps[f"I{i},{j}"].add_tag([f"X{i}", f"Y{j}"])

peps.view_as_(
    qtn.PEPS,
    site_ind_id="k{},{}",
    site_tag_id="I{},{}",
    x_tag_id="X{}",
    y_tag_id="Y{}",
    Lx=Lx,
    Ly=Ly,
)

PEPS(tensors=36, indices=96, Lx=6, Ly=6, max_bond=6)

In [5]:
norm = peps.make_norm()

In [6]:
norm.contract_boundary(
    max_bond=64,
    progbar=True,
    layer_tags=["BRA", "KET"],
)

contracted boundary, Lx=2, Ly=6: : 4it [00:04,  1.04s/it]


1.227912264719253e+46

In [20]:
t = 1.0
V = 8.0
mu = 0.7

terms = {
    (sitea, siteb): sr.fermi_hubbard_spinless_local_array(
        t=t, V=V, mu=mu,
        symmetry="Z2",
        coordinations=(
            site_info[sitea]['coordination'],
            site_info[siteb]['coordination'],
        ),
    ).fuse((0, 1), (2, 3))
    for (sitea, siteb) in peps.gen_bond_coos()
}

In [21]:
ham = qtn.LocalHam2D(Lx, Ly, terms)

In [22]:
su = qtn.SimpleUpdateGen(peps, ham)

In [23]:
# cluster energies may not be accuracte yet
su.evolve(33, tau=0.3)
su.evolve(33, tau=0.1)
su.evolve(33, tau=0.03)
su.evolve(33, tau=0.01)

n=132, tau=0.0100, energy~-13.296717: 100%|##########| 33/33 [00:07<00:00,  4.16it/s]


In [24]:
gs = su.get_state()

In [26]:
gs.compute_local_expectation(
    ham.terms, normalized=True, max_bond=16,
)

-13.685578881751962

In [27]:
gs.compute_local_expectation(
    ham.terms, normalized=True, max_bond=32,
)

-13.17764863509474

In [25]:
gs.compute_local_expectation(
    ham.terms, normalized=True, max_bond=64,
)

-13.24757927850316